In [2]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/NR')
sys.path.append('/home/peitian_zhang/Codes/NR')

import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils.MIND import MIND_iter,MIND_map
from utils.utils import getLoss,getLabel,getId2idx,constructBasicDict,run_eval,run_train
from models.NPA import NPAModel

## setting up the *NPA* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[23\] Npa Neural news recommendation with personalized attention](https://dl.acm.org/doi/abs/10.1145/3292500.3330665)

- *mode*: data to read (*demo*/*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dropout_p*: probability of dropout layer

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

- *user_dim*: user id embedding dimension

- *preference_dim*: user preference embedding dimension

In [3]:
hparams = {
    'mode':'demo',
    'epochs':10,
    'batch_size':100,
    'title_size':30,
    'his_size':50,   
    'npratio':4,
    'dropout_p':0.2,
    'filter_num':400,
    'embedding_dim':300,
    'user_dim':50,
    'preference_dim':200,
    'metrics':'group_auc,mean_mrr,ndcg@5,ndcg@10',
    'gpu':'cuda:0',
    'attrs': ['title']
}

news_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/news.tsv'
news_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/news.tsv'
news_file_pair = (news_file_train,news_file_test)

behavior_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/behaviors.tsv'
behavior_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/behaviors.tsv'
behavior_file_pair = (behavior_file_train,behavior_file_test)

save_path = '/home/peitian_zhang/Codes/NR/models/model_params/NPA_'+ hparams['mode'] +'.model'

if not os.path.exists('data/dictionaries/vocab_{}_{}.pkl'.format(hparams['mode'],'_'.join(hparams['attrs']))):
    constructBasicDict(news_file_pair,behavior_file_pair,hparams['mode'],hparams['attrs'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

dataset_train = MIND_map(hparams=hparams,news_file=news_file_train,behaviors_file=behavior_file_train)

dataset_test = MIND_iter(hparams=hparams,news_file=news_file_test,behaviors_file=behavior_file_test)

vocab = dataset_train.vocab
embedding = GloVe(dim=300,cache='.vector_cache')
vocab.load_vectors(embedding)

# vocab_test = dataset_test.vocab
# vocab_test.load_vectors(embedding)

loader_train = DataLoader(dataset_train,batch_size=hparams['batch_size'],shuffle=True,pin_memory=True,num_workers=3,drop_last=True)
loader_test = DataLoader(dataset_test,batch_size=hparams['batch_size'],pin_memory=True,num_workers=0,drop_last=True)

# writer = SummaryWriter('data/tb/npa/manual/' + datetime.now().strftime("%Y%m%d-%H%M%S"))

In [4]:
npaModel = NPAModel(vocab=vocab,hparams=hparams,uid2idx=dataset_train.uid2index).to(device)
npaModel.train()

NPAModel(
  (wordQueryProject): Linear(in_features=50, out_features=200, bias=True)
  (newsQueryProject): Linear(in_features=50, out_features=200, bias=True)
  (wordPrefProject): Linear(in_features=200, out_features=400, bias=True)
  (newsPrefProject): Linear(in_features=200, out_features=400, bias=True)
  (CNN): Conv1d(300, 400, kernel_size=(3,), stride=(1,), padding=(1,))
  (RELU): ReLU()
  (DropOut): Dropout(p=0.2, inplace=False)
)

In [4]:
if npaModel.training:
    print("training...")
    loss_func = getLoss(npaModel)
    optimizer = optim.Adam(npaModel.parameters(),lr=0.001)
    npaModel = run_train(npaModel,loader_train,optimizer,loss_func,epochs=hparams['epochs'], interval=10)

training...
epoch 0 , step 10 , loss: 1.7048: : 20it [00:04,  4.48it/s]
epoch 1 , step 10 , loss: 1.5742: : 20it [00:04,  4.75it/s]
epoch 2 , step 10 , loss: 1.5362: : 20it [00:03,  5.05it/s]
epoch 3 , step 10 , loss: 1.4253: : 20it [00:04,  4.91it/s]
epoch 4 , step 10 , loss: 1.3535: : 20it [00:04,  4.39it/s]
epoch 5 , step 10 , loss: 1.3083: : 20it [00:04,  4.25it/s]
epoch 6 , step 10 , loss: 1.2820: : 20it [00:04,  4.18it/s]
epoch 7 , step 10 , loss: 1.2278: : 20it [00:04,  4.48it/s]
epoch 8 , step 10 , loss: 1.0816: : 20it [00:04,  4.77it/s]
epoch 9 , step 10 , loss: 1.0422: : 20it [00:04,  4.87it/s]


In [5]:
print("evaluating...")
npaModel.eval()
npaModel.npratio = -1
run_eval(npaModel,loader_test)

0it [00:00, ?it/s]evaluating...
109it [00:16,  6.70it/s]


{'group_auc': 0.5622, 'mean_mrr': 0.2511, 'ndcg@5': 0.2743, 'ndcg@10': 0.3295}